initialize_NLP() was modified from dev version to allow for kwargs...

In [1]:
using NLOptControl
using Plots
using Polynomials
using FastGaussQuadrature

In [2]:
# initialize basic problem definition
ps, nlp = initialize_NLP(numStates=1,numControls=1,Nc=4,Ni=2);

In [3]:
nlp

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: 10
  lengthStateVector: 10
  numControls: 1
  numControlPoints: 8
  lengthControlVector: 8
  lengthDecVector: 20
  stateStartIdx: 1
  stateStopIdx: 10
  controlStartIdx: 11
  controlStopIdx: 18
  timeStartIdx: 19
  timeStopIdx: 20
  stateVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]

In [4]:
ps

NLOptControl.PS_data
  Nc: 4
  Ni: 2
  τ: [-1.0,-0.575319,0.181066,0.822824]
  ω: [0.125,0.657689,0.776387,0.440924]
  t0: 0.0
  tf: 0.0
  stateMatrix: [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]
  controlMatrix: [0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0; 0.0 0.0]

In [5]:
# define problem
α₁ =  -0.3; α₂ = 3; α₃ = -8; α₄ =  7;
@unpack_PS_data ps;
t0 = 0.0;
tf = 10.0;
@pack_PS_data ps;  
t = Array(linspace(t0,tf,100));
####################################
# perform analytical calcualtions -> for plots
####################################
γ = Poly([α₁,α₂,α₁]); #TODO check on that imported binding warning
y = polyval(γ,t);

# evaluate the integral
∫γ = polyint(γ);
Y = polyval(∫γ,t[end]) - polyval(∫γ,t[1]);
C = Y - polyval(∫γ,t[end]); # constant of integration
∫y = polyval(∫γ,t) + C;

# evaluate the derivative
dγ = polyder(γ);
dy = polyval(dγ,t);

##TEMP## -->to get fake optimization data
@unpack_NLP_data nlp;
@unpack_PS_data ps;

τ, ω = gaussradau(Nc); # number of collocation points per interval
# break the problem up into multiple intervals
di, tm, t_data, ωₛ = create_intervals(t0,tf,Ni,Nc,τ,ω); # TODO probably get ride of some of the output

temp = zeros(Float64,1,Ni);
for int in 1:Ni
  temp[int] = di*int;
end
t_dataE = [t_data;temp];

fake_state_data = polyval(γ,t_dataE[:]);
fake_control_data = zeros(lengthControlVector,);
##TEMP## -->to get fake optimization data

8-element Array{Float64,1}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [11]:
# what is coming back is a large vector that has all states and controls --> extract this appropriately
@unpack_NLP_data nlp
decisionVector=[fake_state_data;fake_control_data;t0;tf]; # for now looking at no controls
@pack_NLP_data nlp

function nlp2ocp(decisionVector::Vector{Float64},nlp::NLP_data,ps::PS_data)
    @unpack_PS_data ps
    @unpack_NLP_data nlp  # @unpack_* macro will shadow input argument with the same name as a type-fieldname; see--> http://parametersjl.readthedocs.io/en/latest/manual/#parameters-manual

    # update parameters 
    stateVector = decisionVector[stateStartIdx:stateStopIdx]; 

    controlVector = decisionVector[controlStartIdx:controlStopIdx];
    t0 = decisionVector[timeStartIdx];
    tf = decisionVector[timeStopIdx];
    
    # break up problem to evaluate constraints
    for state in 1:numStates
        for int in 1:Ni-1 # intervals
            if int == 1
                stateMatrix[:,int,state] = stateVector[state:state*numStatePoints];
            else
                stateMatrix[:,int,state] = stateVector[state*int*numStatePoints:(state*int+1)*numStatePoints];
            end
        end
    end
    for control in 1:numControls
        for int in 1:Ni-1 # intervals
            if int == 1 
                controlMatrix[:,int,control] = controlVector[control:control*numControlPoints];
            else
                controlMatrix[:,int,control] = controlVector[control*int*numControlPoints:(control*int+1)*numControlPoints];
            end
        end
    end

    @pack_PS_data ps
    @pack_NLP_data nlp
    return nlp, ps
end

nlp, ps = nlp2ocp(decisionVector,nlp,ps);
#@unpack_PS_data ps
#@unpack_NLP_data nlp
print(nlp,"\n","\n")
#print(ps)

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: 10
  lengthStateVector: 10
  numControls: 1
  numControlPoints: 8
  lengthControlVector: 8
  lengthDecVector: 20
  stateStartIdx: 1
  stateStopIdx: 10
  controlStartIdx: 11
  controlStopIdx: 18
  timeStartIdx: 19
  timeStopIdx: 20
  stateVector: [-0.3,2.54694,5.94253,7.14114,7.2,7.2,6.86184,4.58453,0.969961,-0.3]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [-0.3,2.54694,5.94253,7.14114,7.2,7.2,6.86184,4.58453,0.969961,-0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0]



In [12]:
@pack_NLP_data nlp

NLOptControl.NLP_data
  numStates: 1
  numStatePoints: 10
  lengthStateVector: 10
  numControls: 1
  numControlPoints: 8
  lengthControlVector: 8
  lengthDecVector: 20
  stateStartIdx: 1
  stateStopIdx: 10
  controlStartIdx: 11
  controlStopIdx: 18
  timeStartIdx: 19
  timeStopIdx: 20
  stateVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  controlVector: [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0]
  decisionVector: [-0.3,2.54694,5.94253,7.14114,7.2,7.2,6.86184,4.58453,0.969961,-0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0]